In [161]:
import pandas as pd
import os
import numpy as np
import time
import random
import timeit
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.autograd import Variable
from copy import deepcopy

In [168]:
def filterByValue (sequence, value):
    for thing in sequence:
        if value in thing:
            yield thing

tracknames = os.listdir("train_data/")
tracknames = filterByValue(tracknames, "f-speedway")
data = []

for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    rows = pd.read_csv(path,sep=';', index_col=False)
    data.append(rows)

rawtrain = pd.concat(data)

rawtrain.head()

train_data/oval_f-speedway_1cars_2901757.csv
train_data/oval_f-speedway_1cars_2901718.csv
train_data/oval_f-speedway_1cars_2901726.csv
train_data/oval_f-speedway_1cars_2901749.csv
train_data/oval_f-speedway_1cars_2901742.csv
train_data/oval_f-speedway_1cars_2901730.csv
train_data/oval_f-speedway_1cars_2901737.csv
train_data/oval_f-speedway_1cars_2901755.csv
train_data/oval_f-speedway_1cars_2901739.csv
train_data/oval_f-speedway_1cars_2901752.csv
train_data/oval_f-speedway_1cars_2901734.csv
train_data/oval_f-speedway_1cars_2901746.csv
train_data/oval_f-speedway_1cars_2901744.csv


,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,racePos,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
0,1.0,0,0.0,0.016239,-0.982,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
1,1.0,0,0.0,0.016239,-0.962,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
2,1.0,0,0.0,0.016239,-0.942,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
3,1.0,0,0.0,0.016239,-0.922,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
4,1.0,0,0.0,0.016239,-0.902,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200


In [172]:
# preprocess data
train = deepcopy(rawtrain)

# merge accel and brake into accelbrake
def mergeAccelBrake (row):
    brake = row['brake']
    if brake > 0.0:
        return -1
    else:
        return row['accel']

if not 'accelbrake' in train.columns:
    train.insert(0, 'accelbrake', 0.0, False)

#startTime = timeit.default_timer()
#train['accelbrake'] = train.apply(lambda row: mergeAccelBrake(row), axis=1)
#train = train.apply(lambda row: normalizeSomeColumns(row), axis=1)
train['steer'] /= 2.0
train['steer'] += 0.5
train['speedX'] /= 200.0
train[21:] /= 200.0

#elapsedTime = timeit.default_timer() - startTime
#print("preprocessing took {:.1f}s".format(elapsedTime))
train.iloc[50:60, :]

,accelbrake,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
50,0.0,0.005,0.0,0.002618,0.000090,0.00019,18.39420,0.000006,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
51,0.0,0.005,0.0,0.002629,0.000097,0.00029,18.39420,0.000037,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
52,0.0,0.005,0.0,0.002638,0.000105,0.00039,18.39425,0.000094,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
53,0.0,0.005,0.0,0.002641,0.000113,0.00049,18.39435,0.000179,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
54,0.0,0.005,0.0,0.002656,0.000120,0.00059,18.39445,0.000294,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
55,0.0,0.005,0.0,0.002645,0.000125,0.00069,18.39460,0.000436,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
56,0.0,0.005,0.0,0.002670,0.000124,0.00079,18.39475,0.000602,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
57,0.0,0.005,0.0,0.002679,0.000122,0.00089,18.39495,0.000790,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
58,0.0,0.005,0.0,0.002625,0.000119,0.00099,18.39515,0.000996,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
59,0.0,0.005,0.0,0.002619,0.000118,0.00109,18.39540,0.001219,0.005,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [173]:

inputColumns = list(range(4,5)) + list(range(12,17)) + list(range(21,len(train.columns)))
inp = train.iloc[:,inputColumns]
outp = train.iloc[:,1:4]
print(outp.iloc[0])
print("input columns: {}".format(inp.columns))
print("output columns: {}".format(outp.columns))
input_size = inp.shape[1]
output_size = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,input_size,output_size)

accel    1.0
brake    0.0
steer    0.5
Name: 0, dtype: float64
input columns: Index(['angle', 'speedX', 'speedY', 'speedZ', 'trackPos', 'z', 'track00',
       'track01', 'track02', 'track03', 'track04', 'track05', 'track06',
       'track07', 'track08', 'track09', 'track10', 'track11', 'track12',
       'track13', 'track14', 'track15', 'track16', 'track17', 'track18',
       'oppos00', 'oppos01', 'oppos02', 'oppos03', 'oppos04', 'oppos05',
       'oppos06', 'oppos07', 'oppos08', 'oppos09', 'oppos10', 'oppos11',
       'oppos12', 'oppos13', 'oppos14', 'oppos15', 'oppos16', 'oppos17',
       'oppos18', 'oppos19', 'oppos20', 'oppos21', 'oppos22', 'oppos23',
       'oppos24', 'oppos25', 'oppos26', 'oppos27', 'oppos28', 'oppos29',
       'oppos30', 'oppos31', 'oppos32', 'oppos33', 'oppos34', 'oppos35'],
      dtype='object')
output columns: Index(['accel', 'brake', 'steer'], dtype='object')
145093 61 3


In [174]:
# Hyper Parameters
input_size = inp.shape[1]
hidden_size = 50
output_size = outp.shape[1]
num_epochs = 30
batch_size = 100

In [175]:

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h = F.sigmoid(self.fc1(x))
        h = F.sigmoid(self.fc2(h))
        h = F.sigmoid(self.fc3(h))
        return h

In [32]:
# def minibatch(data, batch_size=32):
#     for i in range(0, len(data), batch_size):
#         yield data[i:i+batch_size]

In [176]:
def get_variable(data, labels = False):
    """Get a Variable given data (pandas dataframe)"""
    # Use float (not double) for better precission: (see reply of smth)
    # https://discuss.pytorch.org/t/problems-with-target-arrays-of-int-int32-types-in-loss-functions/140/3
    
    # Labels need to be long to work (for cross entropy, for MSE they both need to be long)
    if labels:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
    else:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
#     print(tensor)
    return Variable(tensor)

In [139]:
def preprocess(batch):
    """ ... """
    
    inputColumns = list(range(4,5)) + list(range(12,17)) + list(range(21,len(train.columns)))

    x = batch.iloc[:,inputColumns]
    y = batch.iloc[:,1:4]
    
    return x, y

In [177]:
model = Net(input_size, hidden_size, output_size)

In [178]:
#TO CUDA
model.cuda()

Net (
  (fc1): Linear (61 -> 50)
  (fc2): Linear (50 -> 50)
  (fc3): Linear (50 -> 3)
)

In [180]:
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [183]:
for ITER in range(num_epochs):
    
    # Take a random sample of size batchsize
    
    train_loss = 0.0
    start = time.time()
    updates = 0

    for i in range(int(datapoints/(batch_size*5))):
        batch = train.sample(n = batch_size)
        updates += 1

        # pad data with zeros
        x, y = preprocess(batch)
        
        # forward pass
        scores = model(get_variable(x).cuda())
        targets = get_variable(y, labels= True)
        loss = nn.MSELoss()
        output = loss(scores, targets.cuda())
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: avg train loss=%.4f, time=%.2fs" %
          (ITER, train_loss/updates, time.time()-start))

    # evaluate
#     _, _, acc_train = evaluate(model, train)
#     _, _, acc_dev = evaluate(model, dev)
#     print("iter %r: train acc=%.4f  test acc=%.4f" % (ITER, acc_train, acc_dev))

iter 0: avg train loss=0.0034, time=0.91s
iter 1: avg train loss=0.0034, time=0.93s
iter 2: avg train loss=0.0033, time=0.91s
iter 3: avg train loss=0.0032, time=0.90s
iter 4: avg train loss=0.0032, time=0.91s
iter 5: avg train loss=0.0031, time=0.91s
iter 6: avg train loss=0.0031, time=0.91s
iter 7: avg train loss=0.0030, time=0.91s
iter 8: avg train loss=0.0029, time=0.91s
iter 9: avg train loss=0.0029, time=0.92s
iter 10: avg train loss=0.0028, time=0.90s
iter 11: avg train loss=0.0028, time=0.91s
iter 12: avg train loss=0.0027, time=0.91s
iter 13: avg train loss=0.0027, time=0.91s
iter 14: avg train loss=0.0027, time=0.91s
iter 15: avg train loss=0.0026, time=0.92s
iter 16: avg train loss=0.0026, time=0.94s
iter 17: avg train loss=0.0025, time=0.92s
iter 18: avg train loss=0.0025, time=0.91s
iter 19: avg train loss=0.0025, time=0.91s
iter 20: avg train loss=0.0025, time=0.91s
iter 21: avg train loss=0.0024, time=0.91s
iter 22: avg train loss=0.0024, time=0.91s
iter 23: avg train lo

In [185]:
# Test prediction
testval = get_variable(x)
print(testval)
print(model(testval.cuda()))
print(y)

Variable containing:
 3.8664e-04  7.4735e-03 -1.3669e-02  ...   1.0000e+00  1.0000e+00  1.0000e+00
 5.2805e-04  7.4736e-03 -8.4955e-03  ...   1.0000e+00  1.0000e+00  1.0000e+00
-1.9791e-04  7.4730e-03 -1.2743e-02  ...   1.0000e+00  1.0000e+00  1.0000e+00
                ...                   ⋱                   ...                
 4.2511e-04  7.3593e-03 -1.0412e-02  ...   1.0000e+00  1.0000e+00  1.0000e+00
-2.5896e-05  7.4732e-03 -1.8375e-04  ...   1.0000e+00  1.0000e+00  1.0000e+00
-1.2934e-04  7.4754e-03 -5.2904e-03  ...   1.0000e+00  1.0000e+00  1.0000e+00
[torch.FloatTensor of size 100x61]

Variable containing:
1.00000e-02 *
  4.9586  4.7049  4.8348
  4.9495  4.6954  4.8249
  4.9397  4.6731  4.8303
  4.9516  4.6934  4.8320
  4.9504  4.6852  4.8322
  4.9505  4.6852  4.8323
  4.9407  4.6733  4.8314
  4.9446  4.6857  4.8250
  4.9469  4.6833  4.8287
  4.9442  4.6857  4.8243
  4.9447  4.6834  4.8268
  4.9391  4.6726  4.8295
  4.9390  4.6724  4.8295
  4.9455  4.6857  4.8269
  4.9560  4.

In [26]:
save_path_model = 'simple_ff_model_5epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_tanh_tanh_tanh_onlyrelevantdata.pt'
save_path_dict = 'simple_ff_DICT_5epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_tanh_tanh_tanh_onlyrelevantdata.pt'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path_model)
torch.save(model.state_dict(),save_path_dict)
# model.save_state_dict('trainedmodel

/home/jasper-ubuntu/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
